In [1]:
from sklearn import cluster
import numpy as np
from astropy.io import fits
#from astropy.table import Table


#===========================================Load in data=====================================
hdul = fits.open('PCL1002_cat_removeNaN.fits')
data = hdul[1].data
ra = data.field('ra')
dec = data.field('dec')
f250 = data.field('f350')
et250 = data.field('et250')
f350 = data.field('f350')
et350 = data.field('et350')
f500 = data.field('f500')
et500 = data.field('et500')
color_f250_f350_pre = data.field('f250/f350')
color_f350_f500_pre = data.field('f350/f500')

#print(color_f250_f350_pre)
#print(type(ra))
#print(ra.shape)

#color_f250_f350 = []
#color_f350_f500 = []
#for i in range(0, len(color_f250_f350_pre)):
#    if np.isnan(color_f250_f350_pre[i]) == True:
#        color_f250_f350.append(0.0)
#    elif color_f250_f350_pre[i] == float("inf") or color_f250_f350_pre[i] == float("-inf"):
#        color_f250_f350.append(1.0e100)
#    else:
#        color_f250_f350.append(color_f250_f350_pre[i])
#        
#for j in range(0, len(color_f350_f500_pre)):
#    if np.isnan(color_f350_f500_pre[j]) == True:
#        color_f350_f500.append(0.0)
#    elif color_f350_f500_pre[i] == float("inf") or color_f350_f500_pre[i] == float("-inf"):
#        color_f350_f500.append(1.0e100)
#    else:
#        color_f350_f500.append(color_f350_f500_pre[j])
#
#print(color_f250_f350)        
#
X = np.column_stack((ra, dec, f250, et250, f350, et350, f500, et500, color_f250_f350_pre, color_f350_f500_pre))
#X = np.column_stack((ra, dec, f250, et250, f350, et350, f500, et500))
#X = np.column_stack((ra, dec, color_f250_f350_pre, color_f350_f500_pre))

print(X)
print(type(X))
print(X.shape)

#==============================setup classifier=========================================

k_means = cluster.KMeans(n_clusters=3)
k_means.fit(X)

print(k_means.labels_) #Labels of each point. seq[::n] is a sequence of each n-th item in the entire sequence.

np.savetxt('output_kmeans.cat', k_means.labels_, delimiter=' ')

[[149.99773      2.5780058   66.2635     ...   5.077003     0.9692827
    1.3175685 ]
 [150.11905      2.4570513   33.69685    ...   5.012378     2.5274227
    4.430969  ]
 [150.15826      2.139626    65.60122    ...   5.002751     1.2487625
    1.8051956 ]
 ...
 [150.1225       2.1299489    6.2136927  ...   5.0116863    0.69618124
    0.54290056]
 [150.24413      2.4969258    7.772629   ...   5.0401177    1.1416423
    1.1474057 ]
 [149.93184      2.3198414    7.176956   ...   4.991613     0.737808
    0.8937035 ]]
<class 'numpy.ndarray'>
(1206, 10)
[2 2 2 ... 0 0 0]


In [8]:
#---------------------------PCA---------------------------------
from sklearn import cluster
import numpy as np
from astropy.io import fits
from sklearn import decomposition

#===========================================Load in data=====================================
hdul = fits.open('PCL1002_cat_removeNaN.fits')
data = hdul[1].data
ra = data.field('ra')
dec = data.field('dec')
f250 = data.field('f350')
et250 = data.field('et250')
f350 = data.field('f350')
et350 = data.field('et350')
f500 = data.field('f500')
et500 = data.field('et500')
color_f250_f350_pre = data.field('f250/f350')
color_f350_f500_pre = data.field('f350/f500')

X = np.column_stack((ra, dec, f250, et250, f350, et350, f500, et500, color_f250_f350_pre, color_f350_f500_pre))
#X = np.column_stack((ra, dec, f250, et250, f350, et350, f500, et500))
#X = np.column_stack((ra, dec, color_f250_f350_pre, color_f350_f500_pre))

print(X)
print(type(X))
print(X.shape)


pca = decomposition.PCA()
pca.fit(X)
print(pca)

print(pca.explained_variance_)

pca.n_components = 7

X_reduced = pca.fit_transform(X) 
print(X_reduced.shape)

[[149.99773      2.5780058   66.2635     ...   5.077003     0.9692827
    1.3175685 ]
 [150.11905      2.4570513   33.69685    ...   5.012378     2.5274227
    4.430969  ]
 [150.15826      2.139626    65.60122    ...   5.002751     1.2487625
    1.8051956 ]
 ...
 [150.1225       2.1299489    6.2136927  ...   5.0116863    0.69618124
    0.54290056]
 [150.24413      2.4969258    7.772629   ...   5.0401177    1.1416423
    1.1474057 ]
 [149.93184      2.3198414    7.176956   ...   4.991613     0.737808
    0.8937035 ]]
<class 'numpy.ndarray'>
(1206, 10)
PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)
[3.0220724e+02 1.9985861e+02 1.2352040e+01 5.4193509e-01 2.4583198e-02
 1.7281670e-02 1.3899682e-02 4.7092699e-04 8.5868118e-07 2.4558868e-13]
(1206, 7)


In [13]:
#----------------------pipelining--------------------------------------
import numpy as np
from sklearn.decomposition import PCA
#from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import cluster


k_means = cluster.KMeans()
pca = PCA()
pipe = Pipeline(steps=[('pca', pca), ('k_means', k_means)])

#===========================================Load in data=====================================
hdul = fits.open('PCL1002_cat_removeNaN.fits')
data = hdul[1].data
ra = data.field('ra')
dec = data.field('dec')
f250 = data.field('f350')
et250 = data.field('et250')
f350 = data.field('f350')
et350 = data.field('et350')
f500 = data.field('f500')
et500 = data.field('et500')
color_f250_f350_pre = data.field('f250/f350')
color_f350_f500_pre = data.field('f350/f500')

X = np.column_stack((ra, dec, f250, et250, f350, et350, f500, et500, color_f250_f350_pre, color_f350_f500_pre))
#X = np.column_stack((ra, dec, f250, et250, f350, et350, f500, et500))
#X = np.column_stack((ra, dec, color_f250_f350_pre, color_f350_f500_pre))

print(type(X))
print(X.shape)

# Parameters of pipelines can be set using ‘__’ separated parameter names:
param_grid = {
    'pca__n_components': [5, 6, 7, 8, 9, 10],
    'k_means__n_clusters': [2, 3, 4, 5],
    'k_means__init': ['k-means++', 'random'],
}

search = GridSearchCV(pipe, param_grid, iid=False, cv=5)

search.fit(X)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_estimator_)
print(search.best_params_)

#print(k_means.labels_) 

print(search.best_estimator_.named_steps['k_means'].labels_)
np.savetxt('output_pca.cat', search.best_estimator_.named_steps['k_means'].labels_, delimiter=' ')



<class 'numpy.ndarray'>
(1206, 10)
Best parameter (CV score=-66782.352):
Pipeline(memory=None,
         steps=[('pca',
                 PCA(copy=True, iterated_power='auto', n_components=6,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('k_means',
                 KMeans(algorithm='auto', copy_x=True, init='random',
                        max_iter=300, n_clusters=5, n_init=10, n_jobs=None,
                        precompute_distances='auto', random_state=None,
                        tol=0.0001, verbose=0))],
         verbose=False)
{'k_means__init': 'random', 'k_means__n_clusters': 5, 'pca__n_components': 6}
[2 2 2 ... 0 0 0]
